In [24]:
from langchain_openai import ChatOpenAI
from langchain_teddynote import logging
from langchain_teddynote.messages import stream_response
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
import pandas
from llama_parse import LlamaParse
from langchain_community.llms import Ollama  
import streamlit as st
import joblib
from langchain.retrievers import BM25Retriever, EnsembleRetriever
# from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [25]:
embeddings_model = HuggingFaceEmbeddings(  # 768 차원
    model_name='jhgan/ko-sroberta-nli',
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True},
)

c:\Users\jw\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [26]:
llm = ChatOpenAI(
    temperature = 0,
    model_name = "gpt-4o",
)

In [27]:
#Prompt Chain Response function

def PCR(template, docs, query):
    # Prompt
    prompt = ChatPromptTemplate.from_template(template)

    # Chain
    chain = prompt | llm | StrOutputParser()

    # Run
    response = chain.invoke({"question":query, "context":docs})
    return response


# 당신은 광운대학교 학생인 사용자에게 사용자가 입력한 자신의 입학 연도를 기반으로 졸업 요건을 알려주는 인공지능 챗봇입니다.
#             졸업 이수학점 표는 총 7열입니다. 이때 7열중 교양은 (필수+균형)과 기초라는 2열로 다시 나누어집니다.
#             주전공학점(필수 포함)은 단일 전공시와 다전공 이수시로 2열로 나누어집니다.
#             표에서 이웃한 여러 셀이 동일한 값을 가질 경우 그 열의 같은 값을 가지는 이웃한 셀들을 통합하기도 합니다.
#             또한 이 문서는 각 졸업 요건이 동일한 입학 연도 단위 앞 부분에 가., 나., 다. ... 순으로 번호를 매깁니다.
#             이 중 어떤 부분을 참고해서 답변 하였는지 맨 처음에 명시하세요.

In [ ]:
query = "난 소프트웨어학부 2021학년도 입학생이야. 전공 45학점 교양 70학점을 들었어. "

template = '''한국어로 대답할 것. context로 주어진 정보를 바탕으로 query와 비교해 사용자가 졸업하려면 몇 학점이 어느 부분에서 부족한 지 알려주시오.
            오직 자료만을 참고해 최대한 정확하게 말할 것. 어디를 참고했는지 먼저 말할 것. question:{question}, context:{context}'''

# Dense Vector
Dvectorstore = FAISS.load_local(
    './db/faiss',
    embeddings_model,
    distance_strategy=DistanceStrategy.COSINE,
    allow_dangerous_deserialization=True)

Dretriever = Dvectorstore.as_retriever(search_kwargs={'k':3})
Ddocs = Dretriever.get_relevant_documents(query)

# BM25(Keyword Search)
Kdocument = joblib.load("keyword.joblib")
Kretriever = BM25Retriever.from_documents(Kdocument)
Kretriever.k = 3

In [29]:
# Ensemble Retriever
retriever = EnsembleRetriever(
    retrievers = [Kretriever, Dretriever],
    weights = [0.3, 0.7],
)

result = retriever.invoke(query)

print(PCR(template, result, query))

주어진 정보를 바탕으로, 소프트웨어학부 2021학년도 입학생의 졸업 요건을 확인하였습니다. 

출처: Document(id='46906c82-ae16-43cd-94a9-6203bbf9ca64', metadata={'source': './db/files/2025_1_졸업이수요건_19_29.md'})

소프트웨어학부의 졸업 요건은 다음과 같습니다:
- 교양: 필수 22학점 + 균형 21학점 = 총 43학점
- 전공: 60학점
- 졸업이수학점: 133학점

사용자가 이수한 학점:
- 교양: 70학점
- 전공: 45학점

부족한 학점:
- 전공 학점이 60학점 중 45학점만 이수하여 15학점이 부족합니다.
- 총 졸업이수학점은 133학점 중 115학점(교양 70 + 전공 45)만 이수하여 18학점이 부족합니다.

따라서, 졸업을 위해서는 전공 학점 15학점과 총 졸업이수학점 18학점을 추가로 이수해야 합니다.
